In [1]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from twilio.rest import Client

# Set up Twilio credentials (replace these with your own Twilio account details)
TWILIO_SID = 'AC49b47d858501a9a5b2d03f01eb2bb4f2'
TWILIO_AUTH_TOKEN = '73096b1a6180934ffc90e6ad3392ac59'
TWILIO_PHONE_NUMBER = '+14243720494'  # The number you've registered with Twilio
TO_PHONE_NUMBER = '+918169980582'  # Your mobile number to receive the message

# Twilio Client
client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

# Define the event handler for directory monitoring
class MyHandler(FileSystemEventHandler):
    """Custom handler to define actions when events occur."""
    
    def on_modified(self, event):
        """Trigger action when a file is modified."""
        self.send_sms(f'File modified: {event.src_path}')
        
    def on_created(self, event):
        """Trigger action when a new file is created."""
        self.send_sms(f'File created: {event.src_path}')
        
    def on_deleted(self, event):
        """Trigger action when a file is deleted."""
        self.send_sms(f'File deleted: {event.src_path}')
        
    def on_moved(self, event):
        """Trigger action when a file is moved or renamed."""
        self.send_sms(f'File moved: {event.src_path} -> {event.dest_path}')

    def send_sms(self, message):
        """Send an SMS using Twilio"""
        try:
            message = client.messages.create(
                body=message,
                from_=TWILIO_PHONE_NUMBER,
                to=TO_PHONE_NUMBER
            )
            print(f"Message sent: {message.body}")
        except Exception as e:
            print(f"Failed to send message: {str(e)}")

def start_watch(directory):
    """Start watching the specified directory."""
    # Create an event handler
    event_handler = MyHandler()
    
    # Create an observer and assign it the directory to monitor
    observer = Observer()
    observer.schedule(event_handler, path=directory, recursive=True)
    
    # Start the observer
    observer.start()
    
    try:
        while True:
            time.sleep(1)  # Keep the script running to monitor the directory
    except KeyboardInterrupt:
        observer.stop()  # Stop the observer on keyboard interrupt
    observer.join()  # Ensure the observer thread finishes before exiting

if __name__ == "__main__":
    directory_to_watch = "C:/Users/samiksha-shinde/my_new_folder"  # Replace with the directory you want to watch
    print(f"Watching directory: {directory_to_watch}")
    start_watch(directory_to_watch)


Watching directory: C:/Users/samiksha-shinde/my_new_folder
Message sent: Sent from your Twilio trial account - File created: C:/Users/samiksha-shinde/my_new_folder\new\New folder
Message sent: Sent from your Twilio trial account - File modified: C:/Users/samiksha-shinde/my_new_folder\new
Message sent: Sent from your Twilio trial account - File moved: C:/Users/samiksha-shinde/my_new_folder\new\New folder -> C:/Users/samiksha-shinde/my_new_folder\new\ss
Message sent: Sent from your Twilio trial account - File modified: C:/Users/samiksha-shinde/my_new_folder\new
Message sent: Sent from your Twilio trial account - File modified: C:/Users/samiksha-shinde/my_new_folder\new\ss
